Функции для генерации матриц с разными свойствами

In [9]:
import numpy as np
from numba import jit, njit

np.random.seed(1)
n = 1000

# Несимметричная с диагональным преобладанием
@njit
def getA1(n):
    np.random.seed(1)
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            A[i, j] = np.random.rand() * n * (np.abs(i-j) + 1.)**-3
    print(np.linalg.cond(A))
    return A

# Симметричная положительно определенная
@njit
def getA2(n):
    np.random.seed(1)
    A = np.random.rand(n, n)
    A = A.T @ A + n * np.diag(np.random.rand(n) * np.arange(n))
    print(np.linalg.cond(A))
    return A

# Приближенно симметричная 
@njit
def getA3(n):
    np.random.seed(1)
    A = 15 * np.random.rand(n,n)
    A = A + A.T + 2 * np.eye(n)
    
    # Добавляем малое возмущение
    A =+ 1e-1 * np.random.rand(n,n)
    print(np.linalg.cond(A))
    return A

# С блочно-диагональной структурой
@jit
def getA4(n):
    np.random.seed(1)
    A = np.random.rand(n, n) 
    bsize = 10
    nb = n // bsize
    for i in range(nb):
        A[i * bsize : (i+1) * bsize, i * bsize : (i+1) * bsize] = 1. + 100 * np.random.rand(bsize,bsize)
    print(np.linalg.cond(A))
    return A

matrix_list = []

for i, func in enumerate([getA1, getA2, getA3, getA4]):
    A = func(n)
    np.savetxt('A' + str(i+1) + '_matrix.txt', A)
    matrix_list.append(A)

657.688065872041
10304.582741936008
52159.20050622185
74215.39774294924


Идея в том, чтобы студенты применили нужные методы для каждой из матриц: 
* Для $A_1$ - GMRES с предобуславливанием Якоби ($D^-1$)
* Для $A_2$ - CG с предобуславливанием симметричным методом Зейделя или Якоби
* Для $A_3$ - GMRES с каким-либо предобуславливанием, а потом метод, который использует расщепление $A = B + \Delta$, где $B$ -- симметричная, $\Delta$ -- малая по норме матрица
* Для $A_4$ нужно придумать предобуславливатель блочно-диагонального вида:
$$
P^-1\begin{bmatrix}
D_1^{-1} \\
& D_2^{-1} & \\
& & D_n^{-1}
\end{bmatrix}
$$
где $D_i$ - это диагональный блок размера 10 на 10. Его можно обратить прямым методом и на него умножать.